In [1]:
ㅣ!pip install tensorflow
!pip install flask

In [2]:
import tensorflow as tf
import requests
from io import BytesIO
from flask import Flask, request, jsonify
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import numpy as np
from google.colab import drive
import os

drive.mount('/content/gdrive')

# 데이터셋 경로
train_data_dir = '/content/gdrive/My Drive/foods/'
test_data_dir = '/content/gdrive/My Drive/foods/'

# 이미지 사이즈와 클래스 개수
img_width, img_height = 224, 224
num_classes = 5

Mounted at /content/gdrive


In [ ]:
# 데이터 증강 설정 (구도에 대한 파라미터 추가)
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

# 데이터 로딩 및 증강
train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)
app = Flask(__name__)

Found 1906 images belonging to 5 classes.


In [ ]:
# CNN 모델 구성
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
# model.fit(train_generator, epochs=10)

# 모델 저장
#model.save('food_quality_model.h5')
# 모델 불러오기 (학습된 모델을 나중에 재사용할 때 사용)
loaded_model = tf.keras.models.load_model('food_quality_model.h5')

OSError: ignored

In [ ]:
# 이미지 평가 및 재구성 함수
def evaluate_and_reconstruct_image(image_url):
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content))

    image_array = img_to_array(image)
    image_array = np.expand_dims(image_array, axis=0)
    image_array /= 255.0

    # 이미지 예측
    prediction = model.predict(image_array)

    # 카테고리별 피드백 및 점수
    categories = ['구도', '명도', '채도', '선명도', '색온도']
    feedback = []
    scores = []

    for i in range(len(categories)):
        # 각 카테고리에 대한 점수 0부터 10까지
        score = int((prediction[0, i]) * 10)
        scores.append(score)

        # 피드백 메시지 생성
        if i == 0:
            max_index = np.argmax(prediction[0, 1:5])
            shift_directions = ['위쪽', '아래쪽', '왼쪽', '오른쪽']
            feedback.append(f"{categories[i]}: 사진을 {shift_directions[max_index]}으로 움직여 보세요!")
        else:
            if i == 1:
                if score < 0:
                    feedback.append(f"{categories[i]}를 높여주세요!")
                else:
                    feedback.append(f"{categories[i]}를 낮춰주세요!")
            else:
                if score < 0:
                    feedback.append(f"{categories[i]}를 높여주세요!")
                else:
                    feedback.append(f"{categories[i]}를 낮춰주세요!")

    return feedback, scores

In [ ]:
# 이미지 평가 API 엔드포인트
@app.route('/evaluate_image', methods=['POST'])
def evaluate_and_reconstruct_image_endpoint():
    data = request.get_json()
    image_url = data.get('image_url')

    feedback, reconstructed_image_filename = evaluate_and_reconstruct_image(image_url)

    # 재구성된 이미지 파일의 경로 반환
    reconstructed_image_path = os.path.join('static', reconstructed_image_filename)

    return jsonify({'feedback': feedback, 'reconstructed_image_path': reconstructed_image_filename})

# 재구성된 이미지 접근을 위한 엔드포인트
@app.route('/static/<filename>')
def reconstructed_image(filename):
    return send_file(os.path.join('static', filename), mimetype='image/jpeg')

if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
# 저장된 모델 로드
loaded_model = tf.keras.models.load_model('food_quality_model.h5')
loaded_model.summary()